In [139]:
%%bash
#NOTE TO Proofreaders, these are cleanup functions that won't be in the final product
rm -rf simple_lambda
mkdir simple_lambda

#delete all lambda functions with nb1- in the name,  
#all lambda functions in this document are created with the nb1- prefix
#aws lambda list-functions | grep FunctionName | cut -d ":" -f 2| cut -d "," -f 1 | grep "nb1-" xargs -L 1 aws lambda delete-function --function-name
rm -rf simple_package
rm -rf sklearn_package/*

# Deploying the PyData stack onto AWS Lambda

AWS Lambda is a serverless stack from Amazon.  It allows you to have functions that run without needing to maintain a running server.  Lambda functions can be triggered from web requests, SQS, Kenisis, and a variety of other events.  Constructing apps with Lambda functions allows you to easily scale without worrying about spinning up servers.

AWS Lambda has a well known package size limit of 50MB, which can be expanded to 500MB through some hacks.  I hadn't thought it was possible to easily deploy functions that depended on the PyData stack (pandas, numpy, scikit-learn...) because of these size limitations.  In this notebook I wal through

* a simple lambda deployment with no dependencies
* a reqular packaged lambda deployment
* individual steps necessary to dpeloy the PyData stack
* a clean scripted PyData deploy

## Implementation notes about this notebook
I use the python magics of `%%writefile` and `%%bash` extensively.  `%%writefile` allows me to write the lambda functions and bash scripts inline.  `%%bash` allows multiline shell examples.

In a serious dpeloyment system these bash scripts would probably be integrated into Ansible, Chef, or Puppet.  The aws python api could also be used to accomplish the same tasks.  Using the AWS CLI tools through bash is the most straight forward way of experimenting with the lambda platform


## Running this notebook.

The code examples assume a properly configured AWS CLI environment.  The user for the AWS CLI environment must have access to create Lambda functions.  This tutorial also assumes an environment variable AWS_ID with your AWS user_id.  There are scripts integrated which use this variable, and replace the actual account number with "AWS_ID" from the output for privacy.

Some bash commands especially towards the end take a while to run, I have used `time` before these commands.

Finally, running these commands will generate AWS charges, but they should be minimal.

In [99]:
%%writefile aws_sanitize
#!/bin/bash
#this is used to prevent my aws_id leaking into public
#I'm not completely clear why protecting my account number is necessary for security
#but all tutorials do it, so I will too.
replace='$AWS_ID'
sed -e "s/$AWS_ID/$replace/"

Overwriting aws_sanitize


In [100]:
!chmod +x ./aws_sanitize

## Simple Lambda Function

In [3]:
%%writefile simple_lambda/simple_lambda_function.py
import json

def lambda_handler(event, context):
    return {
        'statusCode': 205,
        'body': json.dumps({'event':event, 'context': dir(context),
                            'func_name': 'simpole_lambda_function'})}

Overwriting simple_lambda/simple_lambda_function.py


In [5]:
%%bash
cd simple_lambda
zip function.zip simple_lambda_function.py
aws lambda create-function \
        --function-name simple_lambda_function \
        --handler simple_lambda_function.lambda_handler \
        --zip-file fileb://function.zip \
        --runtime python3.7 \
        --role "arn:aws:iam::$AWS_ID:role/service-role/aws_lambda_role" | ../aws_sanitize

updating: simple_lambda_function.py (deflated 36%)
{
    "FunctionName": "simple_lambda_function",
    "FunctionArn": "arn:aws:lambda:us-east-2:$AWS_ID:function:simple_lambda_function",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::$AWS_ID:role/service-role/aws_lambda_role",
    "Handler": "simple_lambda_function.lambda_handler",
    "CodeSize": 345,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2019-03-31T04:22:26.604+0000",
    "CodeSha256": "3WAwFFRpNc/hkpLnheTZdmeZ2e4N3t4/qap8RdE7HLA=",
    "Version": "$LATEST",
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "9d56b3f7-bc0b-4fd2-9f9a-d3d0238205db"
}


### Let's test the function

In [6]:
%%bash
aws lambda invoke \
    --function-name "simple_lambda_function" \
    --log-type Tail  --invocation-type  RequestResponse slf.out > /dev/null
cat slf.out  | ./aws_sanitize

{"statusCode": 205, "body": "{\"event\": {}, \"context\": [\"__class__\", \"__delattr__\", \"__dict__\", \"__dir__\", \"__doc__\", \"__eq__\", \"__format__\", \"__ge__\", \"__getattribute__\", \"__gt__\", \"__hash__\", \"__init__\", \"__init_subclass__\", \"__le__\", \"__lt__\", \"__module__\", \"__ne__\", \"__new__\", \"__reduce__\", \"__reduce_ex__\", \"__repr__\", \"__setattr__\", \"__sizeof__\", \"__str__\", \"__subclasshook__\", \"__weakref__\", \"_epoch_deadline_time_in_ms\", \"aws_request_id\", \"client_context\", \"function_name\", \"function_version\", \"get_remaining_time_in_millis\", \"identity\", \"invoked_function_arn\", \"log\", \"log_group_name\", \"log_stream_name\", \"memory_limit_in_mb\"], \"func_name\": \"simpole_lambda_function\"}"}


## Simple Package
This example shows how to package simple python libraries with a lambda function

In [7]:
!mkdir simple_package

In [127]:
%%writefile simple_package/package_lambda_function.py
import json
import requests

def lambda_handler(event, context):
    resp = requests.get("https://www.google.com")
    resp_len = len(resp.content)
    return {'resp_len': resp_len}

Overwriting simple_package/package_lambda_function.py


In [128]:
%%bash
cd simple_package
pip install requests --target .  2>&1 | tail -n 5
zip -r9 ./package_function.zip ./* | tail -n 5
aws lambda create-function \
        --function-name pil_lambda_function \
        --handler package_lambda_function.lambda_handler \
        --zip-file fileb://package_function.zip \
        --runtime python3.7 \
        --role "arn:aws:iam::$AWS_ID:role/service-role/aws_lambda_role" | ../aws_sanitize

  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
awscli 1.16.121 has requirement botocore==1.12.111, but you'll have botocore 1.12.112 which is incompatible.
  adding: package/urllib3-1.24.1.dist-info/LICENSE.txt (deflated 41%)
  adding: package/urllib3-1.24.1.dist-info/METADATA (deflated 64%)
  adding: package/urllib3-1.24.1.dist-info/RECORD (deflated 62%)
  adding: package/urllib3-1.24.1.dist-info/top_level.txt (stored 0%)
  adding: package/urllib3-1.24.1.dist-info/WHEEL (deflated 14%)
{
    "FunctionName": "pil_lambda_function",
    "FunctionArn": "arn:aws:lambda:us-east-2:$AWS_ID:function:pil_lambda_function",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::$AWS_ID:role/service-role/aws_lambda_role",
    "Handler": "package_lambda_function.lambda_handler",
    "CodeSize": 1806388,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2019-

### Let's make a script to invoke the lambda function

In [129]:
%%writefile run_function
#!/bin/bash
aws lambda invoke \
    --function-name $1 \
    --log-type Tail  --invocation-type  RequestResponse slf.out > /dev/null
cat slf.out  | ./aws_sanitize

Overwriting run_function


In [130]:
!chmod +x run_function
!./run_function pil_lambda_function

{"resp_len": 11180}


## PyData Package with Pandas and Scikit-Learn

In [131]:
!mkdir sklearn_package

mkdir: sklearn_package: File exists


In [132]:
%%writefile sklearn_package/package_lambda_function.py
import json
import pandas
import sklearn

def lambda_handler(event, context):
    return {
        'statusCode': 205,
        'body': json.dumps({'event':event, 'context': dir(context),
                            'func_name': 'package_lambda_function'})}

Writing sklearn_package/package_lambda_function.py


In [35]:
%%bash
#note the extra options to force linux packages even if you are on OS X
pip install pandas scikit-learn \
        --platform manylinux1_x86_64\
        --python-version 37 \
        --only-binary=:all:  --target sklearn_package 2>&1 | tail -n 5

  Using cached https://files.pythonhosted.org/packages/3e/7e/5cee36eee5b3194687232f6150a89a38f784883c612db7f4da2ab190980d/scipy-1.2.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl


In [36]:
%%bash
cd sklearn_package
zip -r9 sklearn_package.zip ./ | tail -n 3

  adding: sklearn/utils/tests/test_sparsefuncs.py (deflated 84%)
  adding: sklearn/utils/tests/test_stats.py (deflated 58%)
  adding: sklearn/utils/tests/test_testing.py (deflated 80%)
  adding: sklearn/utils/tests/test_utils.py (deflated 73%)
  adding: sklearn/utils/tests/test_validation.py (deflated 77%)
  adding: sklearn/utils/validation.py (deflated 74%)
  adding: sklearn/utils/weight_vector.cpython-37m-x86_64-linux-gnu.so (deflated 58%)


In [37]:
%%bash
cd sklearn_package
time aws lambda create-function \
        --function-name sklearn_lambda_function \
        --handler package_lambda_function.lambda_handler \
        --zip-file fileb://sklearn_package.zip \
        --runtime python3.7 \
        --role "arn:aws:iam::$AWS_ID:role/service-role/aws_lambda_role" | ../aws_sanitize


Connection was closed before we received a valid response from endpoint URL: "https://lambda.us-east-2.amazonaws.com/2015-03-31/functions".

real	5m14.732s
user	0m4.530s
sys	0m3.997s


## the above failed because the pacakge was too big
Instead we have to upload the package to s3, then point he lambda function at the s3 package

In [41]:
%%bash
cd sklearn_package
aws s3 mb s3://pandas-sklearn-demo  2>&1 > /dev/null
time aws s3 cp sklearn_package/sklearn_package.zip s3://pandas-sklearn-demo/sklearn_package.zip
aws lambda update-function-code \
        --function-name pandas_sklearn_demo2 \
        --s3-bucket pandas-sklearn-demo \
        --s3-key sklearn_package.zip | ../aws_sanitize


{
    "FunctionName": "pandas_sklearn_demo2",
    "FunctionArn": "arn:aws:lambda:us-east-2:$AWS_ID:function:pandas_sklearn_demo2",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::$AWS_ID:role/service-role/aws_lambda_role",
    "Handler": "pandas_sklearn_demo.lambda_handler",
    "CodeSize": 70547257,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2019-04-03T00:45:21.232+0000",
    "CodeSha256": "QkXoj0DXvQJ3kk5RhPbUfUWuWvtE4m7xXlI8qAW9/7s=",
    "Version": "$LATEST",
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "f2fd6408-96b0-46b7-91ab-67e0d2a920f9"
}



real	0m11.197s
user	0m0.490s
sys	0m0.094s


## create_lambda script
Packaging up these lambda function is getting complex.  let's put all of this into a script

In [125]:
%%writefile create_lambda
#!/bin/bash

#note function name must be the same as the module name
function_name=$1
ver_number=$2
package_zip=$3

ver_name="${function_name}_${ver_number}"
mod_file="${function_name}.py"
mod_name=$function_name
handler_name="${mod_name}.lambda_handler"
empty_zip_file=/tmp/empty.zip
bucket_zip="${versioned_name}.zip"

zip_file=/tmp/function.zip
cp $package_zip $zip_file
zip $zip_file $mod_file
zip $empty_zip_file $mod_file

#we use the empty_zip file because this will fail with a larger zip and the s3 upload will have to be run again
#anyway,  this will save some time.
aws lambda create-function --function-name $ver_name \
           --zip-file fileb://$empty_zip_file  --handler $handler_name\
           --runtime python3.7 \
           --role "arn:aws:iam::$AWS_ID:role/service-role/aws_lambda_role" > /dev/null

#this bucket is hardcoded.  this can probably be improved upon
#aws s3 mb s3://pandas-sklearn-demo  2>&1 > /dev/null
aws s3 cp $zip_file  s3://pandas-sklearn-demo/$bucket_zip > /dev/null

aws lambda update-function-code \
        --s3-bucket pandas-sklearn-demo \
        --s3-key $bucket_zip \
        --function-name $ver_name | ./aws_sanitize
aws lambda update-function-configuration \
        --function-name $ver_name \
        --timeout 10 | ./aws_sanitize



./run_function $ver_name

Overwriting create_lambda


## Now we have to make bare package zip directories

In [52]:
%%bash
chmod +x create_lambda
cd sklearn_package

#zip up the sklearn package
zip -r9 ../sklearn_package.zip ./ | tail -n 3

  adding: sklearn/utils/tests/test_validation.py (deflated 77%)
  adding: sklearn/utils/validation.py (deflated 74%)
  adding: sklearn/utils/weight_vector.cpython-37m-x86_64-linux-gnu.so (deflated 58%)


## sklearn_example2

In [134]:
%%writefile sklearn_example2.py
import json
import pandas as pd

def lambda_handler(event, context):
    df = pd.DataFrame({'a':[5,3,2,10], 'b':[20, 30, 40, 50]})
    return {'mean':  repr(df.mean())}


Overwriting sklearn_example2.py


In [86]:
!time ./create_lambda sklearn_example2 6 sklearn_package.zip

function_name sklearn_example2
ver_name sklearn_example2_6
mod_name sklearn_example2
bucket_zip .zip
  adding: sklearn_example2.py (deflated 26%)
updating: sklearn_example2.py (deflated 26%)
{
    "FunctionName": "sklearn_example2_6",
    "FunctionArn": "arn:aws:lambda:us-east-2:$AWS_ID:function:sklearn_example2_6",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::$AWS_ID:role/service-role/aws_lambda_role",
    "Handler": "sklearn_example2.lambda_handler",
    "CodeSize": 378,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2019-04-03T03:12:24.053+0000",
    "CodeSha256": "B2egszZdhKTDmdDLliRvM2ftaZ5g49I9Cpjt/bY4PQc=",
    "Version": "$LATEST",
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "3b7a70a8-9784-47c5-a18b-71a92f863866"
}
upload: ../../../../tmp/function.zip to s3://pandas-sklearn-demo/.zip
{
    "FunctionName": "sklearn_example2_6",
    "FunctionArn": "arn:aws:lambda:us-east-2:$AWS_ID:function:sklearn_ex

## Matplotlib example

In [110]:
%%bash
#note the extra options to force linux packages even if you are on OS X
mkdir pydata_full
rm -rf pydata_full/*
pip install pandas matplotlib \
        --platform manylinux1_x86_64\
        --python-version 37 \
        --only-binary=:all:  --target pydata_full 2>&1 | tail -n 5

pip install papermill jupyter \
        --target pydata_full 2>&1 | tail -n 5
rm pydata_full.zip
cd pydata_full
zip -r9 ../pydata_full.zip ./ | tail -n 3

  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d1/6a/4b2fcefd2ea0868810e92d519dacac1ddc64a2e53ba9e3422c3b62b378a6/setuptools-40.8.0-py2.py3-none-any.whl
Target directory /Users/paddy/code/lambda_py_notebooks/pydata_full/setuptools already exists. Specify --upgrade to force replacement.
Target directory /Users/paddy/code/lambda_py_notebooks/pydata_full/setuptools-40.8.0.dist-info already exists. Specify --upgrade to force replacement.
Target directory /Users/paddy/code/lambda_py_notebooks/pydata_full/six-1.12.0.dist-info already exists. Specify --upgrade to force replacement.
Target directory /Users/paddy/code/lambda_py_notebooks/pydata_full/six.py already exists. Specify --upgrade to force replacement.
Target directory /Users/paddy/code/lambda_py_notebooks/pydata_full/bin already exists. Specify --upgrade to force replacement.

mkdir: pydata_full: File exists


In [119]:
%%writefile mpl_debug.py
import json

from io import StringIO, BytesIO

import matplotlib as mpl
import matplotlib.pyplot as plt

import boto3
import botocore

def save_plot(fig, bucket='pandas-sklearn-demo', key='plot.png'):
    buffer_ = BytesIO()
    fig.savefig(buffer_)
    buffer_.seek(0)

    
    s3 = boto3.resource('s3')
    bucket_obj = s3.Bucket(bucket)
    
    bucket_obj.put_object(
        Key=key, Body=buffer_,
        StorageClass='REDUCED_REDUNDANCY',
        #ACL='public-read',
        ContentType='image/png')
    s3Client = boto3.client('s3')
    temp_url = s3Client.generate_presigned_url(
        'get_object', Params = {'Bucket': bucket, 'Key': key}, ExpiresIn = 100)
    return temp_url

    
def lambda_handler(event, context):
    mpl.use('agg')

    fig, ax = plt.subplots(figsize=(10,7))
    ax.plot(range(20), range(20))
    image_url = save_plot(fig, key='plot7.png')
    return {'image_url': image_url}



Overwriting mpl_debug.py


In [126]:
!time ./create_lambda mpl_debug 10 pydata_full.zip
!say "beep"

  adding: mpl_debug.py (deflated 46%)
updating: mpl_debug.py (deflated 46%)
{
    "FunctionName": "mpl_debug_10",
    "FunctionArn": "arn:aws:lambda:us-east-2:$AWS_ID:function:mpl_debug_10",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::$AWS_ID:role/service-role/aws_lambda_role",
    "Handler": "mpl_debug.lambda_handler",
    "CodeSize": 76537816,
    "Description": "",
    "Timeout": 3,
    "MemorySize": 128,
    "LastModified": "2019-04-03T17:50:16.372+0000",
    "CodeSha256": "o/sLJ4/L5z6Ha9lGiFD8HvX9j37OgzfbdOSZ7E7pIRs=",
    "Version": "$LATEST",
    "TracingConfig": {
        "Mode": "PassThrough"
    },
    "RevisionId": "08acd50c-dd0b-4cd5-91ce-6889076b188c"
}
{
    "FunctionName": "mpl_debug_10",
    "FunctionArn": "arn:aws:lambda:us-east-2:$AWS_ID:function:mpl_debug_10",
    "Runtime": "python3.7",
    "Role": "arn:aws:iam::$AWS_ID:role/service-role/aws_lambda_role",
    "Handler": "mpl_debug.lambda_handler",
    "CodeSize": 76537816,
    "Description": "",
    "Timeo

In [117]:
!time ./run_function mpl_debug_6

{"img_len": 26585}

real	0m6.067s
user	0m0.878s
sys	0m0.242s


In [ ]:
#package via lambda, should be much faster.  just upload file, rezip in lambda function


In [140]:
!ls

AWS Lambda walkthrough-v2.ipynb pydata_full.zip
AWS Lambda walkthrough.ipynb    run_function
README.md                       simple_lambda
Untitled.ipynb                  simple_package
aws_sanitize                    sklearn_example2.py
create_lambda                   sklearn_package
foo.txt                         sklearn_package.zip
mpl_debug.py                    slf.out
pydata_full


In [141]:
!pwd

/Users/paddy/code/lambda_py_notebooks


In [142]:
!ls sklearn_package/